In [1]:
# Retry: robust dataset generation with installation fallback for Faker
import sys, subprocess, os, uuid, random
from datetime import datetime, timedelta

# install faker if missing
try:
    from faker import Faker
except Exception:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "faker"])
    from faker import Faker

import pandas as pd
Faker = Faker
fake = Faker()
Faker.seed(42)
random.seed(42)

OUT_DIR = "/mnt/data/estate_sim"
os.makedirs(OUT_DIR, exist_ok=True)

# Parameters
NUM_HOUSES = 10
LANDLORDS_COUNT = 10
MEMBERS_TOTAL = 36
START_JOIN = datetime(2020,1,1)
END_JOIN = datetime(2024,6,1)
CHARGE_MONTH = datetime(2024,6,1)
SECURITY_UTILITY_ID = "UT004"

def house_id(n):
    return f"H{n:03d}"

def apartment_id(house_num, unit):
    return f"APT-{house_num:03d}{unit:02d}"

def member_code(n):
    return f"M{n:03d}"

def charge_code(n):
    return f"CH{n:03d}"

def payment_code(n):
    return f"P{n:04d}"

# Houses
houses = [{"HouseID": house_id(i), "HouseNumber": i} for i in range(1, NUM_HOUSES+1)]
houses_df = pd.DataFrame(houses)

# Members: landlords first
members = []
landlord_ids = []
for i in range(1, LANDLORDS_COUNT+1):
    mid = member_code(i)
    h = house_id(i)
    name = fake.name()
    email = f"{name.lower().replace(' ','.')}@estate.com"
    phone = fake.msisdn()[:13]
    join = fake.date_between(START_JOIN, END_JOIN)
    is_resident = random.choice([True, False])
    membership_status = random.choices(["Current","Former"], weights=[0.9,0.1])[0]
    members.append({
        "MemberID": mid,
        "FullName": name,
        "Role": "Landlord",
        "PhoneNumber": phone,
        "Email": email,
        "JoiningDate": join,
        "HouseID": h,
        "ApartmentID": "",
        "IsResident": is_resident,
        "MembershipStatus": membership_status
    })
    landlord_ids.append(mid)

next_id = LANDLORDS_COUNT + 1
num_tenants = 18
num_family = MEMBERS_TOTAL - LANDLORDS_COUNT - num_tenants

tenant_candidates = []
for i in range(num_tenants):
    mid = member_code(next_id)
    name = fake.name()
    email = f"{name.lower().replace(' ','.')}@estate.com"
    phone = fake.msisdn()[:13]
    join = fake.date_between(START_JOIN, END_JOIN)
    members.append({
        "MemberID": mid,
        "FullName": name,
        "Role": "Tenant",
        "PhoneNumber": phone,
        "Email": email,
        "JoiningDate": join,
        "HouseID": None,
        "ApartmentID": "",
        "IsResident": False,
        "MembershipStatus": random.choices(["Current","Former"], weights=[0.85,0.15])[0]
    })
    tenant_candidates.append(mid)
    next_id += 1

for i in range(num_family):
    mid = member_code(next_id)
    name = fake.name()
    email = f"{name.lower().replace(' ','.')}@estate.com"
    phone = fake.msisdn()[:13]
    join = fake.date_between(START_JOIN, END_JOIN)
    house_assigned = house_id(random.randint(1, NUM_HOUSES))
    members.append({
        "MemberID": mid,
        "FullName": name,
        "Role": "Family",
        "PhoneNumber": phone,
        "Email": email,
        "JoiningDate": join,
        "HouseID": house_assigned,
        "ApartmentID": "",
        "IsResident": True,
        "MembershipStatus": "Current"
    })
    next_id += 1

members_df = pd.DataFrame(members)

# Landlords derived table and tenant counts
landlords = []
assignment = ["Fully Owned","Partly Rented","Fully Owned","Fully Rented","Fully Owned",
              "Partly Rented","Partly Rented","Fully Rented","Fully Rented","Partly Rented"]

for idx, lid in enumerate(landlord_ids):
    house = house_id(idx+1)
    row = members_df.loc[members_df.MemberID==lid].iloc[0]
    is_res = row["IsResident"]
    membership_status = row["MembershipStatus"]
    ownership = assignment[idx]
    if ownership=="Fully Owned":
        num_tenants = 0
    elif ownership=="Partly Rented":
        num_tenants = random.randint(1,3)
    else:
        num_tenants = random.randint(4,5)
    if membership_status in ("Former",):
        ownership = "Fully Rented"
        num_tenants = random.randint(4,5)
    landlords.append({
        "LandlordID": lid,
        "FullName": row["FullName"],
        "PhoneNumber": row["PhoneNumber"],
        "Email": row["Email"],
        "HouseID": house,
        "StreetAddress": fake.street_address(),
        "IsResident": is_res,
        "OwnershipStatus": ownership,
        "MembershipStatus": membership_status,
        "NumberOfTenants": num_tenants
    })

landlords_df = pd.DataFrame(landlords)

# Allocate tenants per landlord
tenants = []
tenant_index = 0
apt_counters = {i:0 for i in range(1, NUM_HOUSES+1)}
for ld in landlords:
    h = ld["HouseID"]
    hnum = int(h[1:])
    n = ld["NumberOfTenants"]
    if n == 0:
        continue
    for unit in range(n):
        if tenant_index >= len(tenant_candidates):
            break
        tid = tenant_candidates[tenant_index]
        tenant_index += 1
        apt_counters[hnum] += 1
        apt_id = apartment_id(hnum, apt_counters[hnum])
        members_df.loc[members_df.MemberID==tid, "HouseID"] = h
        members_df.loc[members_df.MemberID==tid, "ApartmentID"] = apt_id
        lease_start = fake.date_between(datetime(2022,1,1), datetime(2024,5,1))
        lease_end = lease_start + timedelta(days=365)
        tenancy_status = random.choices(["Active","Former"], weights=[0.85,0.15])[0]
        tenants.append({
            "TenantID": tid,
            "FullName": members_df.loc[members_df.MemberID==tid, "FullName"].values[0],
            "HouseID": h,
            "ApartmentID": apt_id,
            "LeaseStart": lease_start,
            "LeaseEnd": lease_end,
            "TenantResidencyStatus": tenancy_status,
            "LandlordID": ld["LandlordID"]
        })

# If remaining tenants, assign to random partly/fully rented houses
if tenant_index < len(tenant_candidates):
    remaining = tenant_candidates[tenant_index:]
    candidate_houses = [l for l in landlords if l["NumberOfTenants"]>0]
    for tid in remaining:
        ld = random.choice(candidate_houses)
        h = ld["HouseID"]
        hnum = int(h[1:])
        apt_counters[hnum] += 1
        apt_id = apartment_id(hnum, apt_counters[hnum])
        members_df.loc[members_df.MemberID==tid, "HouseID"] = h
        members_df.loc[members_df.MemberID==tid, "ApartmentID"] = apt_id
        lease_start = fake.date_between(datetime(2022,1,1), datetime(2024,5,1))
        lease_end = lease_start + timedelta(days=365)
        tenancy_status = random.choices(["Active","Former"], weights=[0.85,0.15])[0]
        tenants.append({
            "TenantID": tid,
            "FullName": members_df.loc[members_df.MemberID==tid, "FullName"].values[0],
            "HouseID": h,
            "ApartmentID": apt_id,
            "LeaseStart": lease_start,
            "LeaseEnd": lease_end,
            "TenantResidencyStatus": tenancy_status,
            "LandlordID": ld["LandlordID"]
        })

tenants_df = pd.DataFrame(tenants)

# Utilities
utilities = [
    {"UtilityID": "UT001", "UtilityName": "Electricity", "Description":"Monthly electricity charge", "BillingFrequency":"Monthly"},
    {"UtilityID": "UT002", "UtilityName": "Water", "Description":"Monthly water charge", "BillingFrequency":"Monthly"},
    {"UtilityID": "UT003", "UtilityName": "Waste Disposal", "Description":"Monthly waste charge", "BillingFrequency":"Monthly"},
    {"UtilityID": "UT004", "UtilityName": "Security", "Description":"Monthly security levy (per apartment/individual)", "BillingFrequency":"Monthly"},
    {"UtilityID": "UT005", "UtilityName": "Development Levy", "Description":"One-time lump-sum development levy", "BillingFrequency":"One-time"}
]
utilities_df = pd.DataFrame(utilities)

# Utility charges
charges = []
charge_counter = 1
for hrow in houses_df.itertuples():
    h = hrow.HouseID
    house_tenants = tenants_df[tenants_df.HouseID==h]
    for ut in utilities:
        uid = ut["UtilityID"]
        if uid == SECURITY_UTILITY_ID:
            # landlord-level security charge
            expected_amount = random.randint(4000,8000)
            charges.append({
                "ChargeID": charge_code(charge_counter),
                "UtilityID": uid,
                "HouseID": h,
                "ApartmentID": "",
                "ChargeAmount": float(expected_amount),
                "ChargeDate": CHARGE_MONTH.date(),
                "DueDate": (CHARGE_MONTH + timedelta(days=29)).date(),
                "ChargeMonth": CHARGE_MONTH.strftime("%Y-%m"),
                "PaymentStatus": "Pending"
            })
            charge_counter += 1
            for t in house_tenants.itertuples():
                expected_amount = random.randint(3000,7000)
                charges.append({
                    "ChargeID": charge_code(charge_counter),
                    "UtilityID": uid,
                    "HouseID": h,
                    "ApartmentID": t.ApartmentID,
                    "ChargeAmount": float(expected_amount),
                    "ChargeDate": CHARGE_MONTH.date(),
                    "DueDate": (CHARGE_MONTH + timedelta(days=29)).date(),
                    "ChargeMonth": CHARGE_MONTH.strftime("%Y-%m"),
                    "PaymentStatus": "Pending"
                })
                charge_counter += 1
        else:
            expected_amount = random.randint(8000,22000) if uid!="UT002" else random.randint(3000,8000)
            charges.append({
                "ChargeID": charge_code(charge_counter),
                "UtilityID": uid,
                "HouseID": h,
                "ApartmentID": "",
                "ChargeAmount": float(expected_amount),
                "ChargeDate": CHARGE_MONTH.date(),
                "DueDate": (CHARGE_MONTH + timedelta(days=29)).date(),
                "ChargeMonth": CHARGE_MONTH.strftime("%Y-%m"),
                "PaymentStatus": "Pending"
            })
            charge_counter += 1

charges_df = pd.DataFrame(charges)

# Payments simulation (~70% charges get payments)
payments = []
pay_counter = 1
for ch in charges_df.itertuples():
    if random.random() < 0.7:
        full_pay = random.random() < 0.6
        if full_pay:
            paid_amount = ch.ChargeAmount
        else:
            paid_amount = round(ch.ChargeAmount * random.uniform(0.3,0.9),2)
        payments.append({
            "PaymentID": payment_code(pay_counter),
            "ChargeID": ch.ChargeID,
            "HouseID": ch.HouseID,
            "ApartmentID": ch.ApartmentID,
            "PaidAmount": float(paid_amount),
            "PaymentDate": (CHARGE_MONTH + timedelta(days=random.randint(0,25))).date(),
            "PaymentMethod": random.choice(["Paystack","Bank Transfer","POS","Cash"]),
            "ReferenceNo": str(uuid.uuid4())[:18],
            "UtilityID": ch.UtilityID,
            "ExpectedAmount": ch.ChargeAmount
        })
        pay_counter += 1

payments_df = pd.DataFrame(payments)

# Update charges PaymentStatus as trigger would do
paid_sums = payments_df.groupby("ChargeID")["PaidAmount"].sum().reset_index().rename(columns={"PaidAmount":"TotalPaid"})
charges_df = charges_df.merge(paid_sums, how="left", on="ChargeID")
charges_df["TotalPaid"] = charges_df["TotalPaid"].fillna(0.0)
def status_from_amount(row):
    if row["TotalPaid"] >= row["ChargeAmount"]:
        return "Paid"
    elif row["TotalPaid"] > 0:
        return "Partially Paid"
    else:
        return "Pending"
charges_df["PaymentStatus"] = charges_df.apply(status_from_amount, axis=1)
charges_df = charges_df.drop(columns=["TotalPaid"])

# Ensure membership landlord house assignments
for ld in landlords_df.itertuples():
    members_df.loc[members_df.MemberID==ld.LandlordID, "HouseID"] = ld.HouseID
    members_df.loc[members_df.MemberID==ld.LandlordID, "ApartmentID"] = ""

# Save CSVs
members_df.to_csv(os.path.join(OUT_DIR,"membership.csv"), index=False)
landlords_df.to_csv(os.path.join(OUT_DIR,"landlords.csv"), index=False)
tenants_df.to_csv(os.path.join(OUT_DIR,"tenants.csv"), index=False)
utilities_df.to_csv(os.path.join(OUT_DIR,"utilities.csv"), index=False)
charges_df.to_csv(os.path.join(OUT_DIR,"utility_charges.csv"), index=False)
payments_df.to_csv(os.path.join(OUT_DIR,"payments.csv"), index=False)

# Try to display dataframes using caas_jupyter_tools if available
try:
    import caas_jupyter_tools as cjt
    cjt.display_dataframe_to_user("Membership (preview)", members_df.head(15))
    cjt.display_dataframe_to_user("Landlords (preview)", landlords_df.head(15))
    cjt.display_dataframe_to_user("Tenants (preview)", tenants_df.head(15))
    cjt.display_dataframe_to_user("Utility Charges (preview)", charges_df.head(20))
    cjt.display_dataframe_to_user("Payments (preview)", payments_df.head(20))
except Exception:
    pass

print("Saved CSVs to", OUT_DIR)
for f in sorted(os.listdir(OUT_DIR)):
    print("-", f)


CalledProcessError: Command '['C:\\Users\\DELL\\anaconda3\\python.exe', '-m', 'pip', 'install', 'faker']' returned non-zero exit status 1.

In [3]:
!pip install Faker

ERROR: Could not install packages due to an OSError: Could not find a suitable TLS CA certificate bundle, invalid path: C:\Program Files\PostgreSQL\17\ssl\certs\ca-bundle.crt



In [5]:
# Retry: robust dataset generation with installation fallback for Faker
import sys, subprocess, os, uuid, random
from datetime import datetime, timedelta

# install faker if missing
try:
    from faker import Faker
except Exception:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "faker"])
    from faker import Faker

import pandas as pd
Faker = Faker
fake = Faker()
Faker.seed(42)
random.seed(42)

OUT_DIR = "/mnt/data/estate_sim"
os.makedirs(OUT_DIR, exist_ok=True)

# Parameters
NUM_HOUSES = 10
LANDLORDS_COUNT = 10
MEMBERS_TOTAL = 36
START_JOIN = datetime(2020,1,1)
END_JOIN = datetime(2024,6,1)
CHARGE_MONTH = datetime(2024,6,1)
SECURITY_UTILITY_ID = "UT004"

def house_id(n):
    return f"H{n:03d}"

def apartment_id(house_num, unit):
    return f"APT-{house_num:03d}{unit:02d}"

def member_code(n):
    return f"M{n:03d}"

def charge_code(n):
    return f"CH{n:03d}"

def payment_code(n):
    return f"P{n:04d}"

# Houses
houses = [{"HouseID": house_id(i), "HouseNumber": i} for i in range(1, NUM_HOUSES+1)]
houses_df = pd.DataFrame(houses)

# Members: landlords first
members = []
landlord_ids = []
for i in range(1, LANDLORDS_COUNT+1):
    mid = member_code(i)
    h = house_id(i)
    name = fake.name()
    email = f"{name.lower().replace(' ','.')}@estate.com"
    phone = fake.msisdn()[:13]
    join = fake.date_between(START_JOIN, END_JOIN)
    is_resident = random.choice([True, False])
    membership_status = random.choices(["Current","Former"], weights=[0.9,0.1])[0]
    members.append({
        "MemberID": mid,
        "FullName": name,
        "Role": "Landlord",
        "PhoneNumber": phone,
        "Email": email,
        "JoiningDate": join,
        "HouseID": h,
        "ApartmentID": "",
        "IsResident": is_resident,
        "MembershipStatus": membership_status
    })
    landlord_ids.append(mid)

next_id = LANDLORDS_COUNT + 1
num_tenants = 18
num_family = MEMBERS_TOTAL - LANDLORDS_COUNT - num_tenants

tenant_candidates = []
for i in range(num_tenants):
    mid = member_code(next_id)
    name = fake.name()
    email = f"{name.lower().replace(' ','.')}@estate.com"
    phone = fake.msisdn()[:13]
    join = fake.date_between(START_JOIN, END_JOIN)
    members.append({
        "MemberID": mid,
        "FullName": name,
        "Role": "Tenant",
        "PhoneNumber": phone,
        "Email": email,
        "JoiningDate": join,
        "HouseID": None,
        "ApartmentID": "",
        "IsResident": False,
        "MembershipStatus": random.choices(["Current","Former"], weights=[0.85,0.15])[0]
    })
    tenant_candidates.append(mid)
    next_id += 1

for i in range(num_family):
    mid = member_code(next_id)
    name = fake.name()
    email = f"{name.lower().replace(' ','.')}@estate.com"
    phone = fake.msisdn()[:13]
    join = fake.date_between(START_JOIN, END_JOIN)
    house_assigned = house_id(random.randint(1, NUM_HOUSES))
    members.append({
        "MemberID": mid,
        "FullName": name,
        "Role": "Family",
        "PhoneNumber": phone,
        "Email": email,
        "JoiningDate": join,
        "HouseID": house_assigned,
        "ApartmentID": "",
        "IsResident": True,
        "MembershipStatus": "Current"
    })
    next_id += 1

members_df = pd.DataFrame(members)

# Landlords derived table and tenant counts
landlords = []
assignment = ["Fully Owned","Partly Rented","Fully Owned","Fully Rented","Fully Owned",
              "Partly Rented","Partly Rented","Fully Rented","Fully Rented","Partly Rented"]

for idx, lid in enumerate(landlord_ids):
    house = house_id(idx+1)
    row = members_df.loc[members_df.MemberID==lid].iloc[0]
    is_res = row["IsResident"]
    membership_status = row["MembershipStatus"]
    ownership = assignment[idx]
    if ownership=="Fully Owned":
        num_tenants = 0
    elif ownership=="Partly Rented":
        num_tenants = random.randint(1,3)
    else:
        num_tenants = random.randint(4,5)
    if membership_status in ("Former",):
        ownership = "Fully Rented"
        num_tenants = random.randint(4,5)
    landlords.append({
        "LandlordID": lid,
        "FullName": row["FullName"],
        "PhoneNumber": row["PhoneNumber"],
        "Email": row["Email"],
        "HouseID": house,
        "StreetAddress": fake.street_address(),
        "IsResident": is_res,
        "OwnershipStatus": ownership,
        "MembershipStatus": membership_status,
        "NumberOfTenants": num_tenants
    })

landlords_df = pd.DataFrame(landlords)

# Allocate tenants per landlord
tenants = []
tenant_index = 0
apt_counters = {i:0 for i in range(1, NUM_HOUSES+1)}
for ld in landlords:
    h = ld["HouseID"]
    hnum = int(h[1:])
    n = ld["NumberOfTenants"]
    if n == 0:
        continue
    for unit in range(n):
        if tenant_index >= len(tenant_candidates):
            break
        tid = tenant_candidates[tenant_index]
        tenant_index += 1
        apt_counters[hnum] += 1
        apt_id = apartment_id(hnum, apt_counters[hnum])
        members_df.loc[members_df.MemberID==tid, "HouseID"] = h
        members_df.loc[members_df.MemberID==tid, "ApartmentID"] = apt_id
        lease_start = fake.date_between(datetime(2022,1,1), datetime(2024,5,1))
        lease_end = lease_start + timedelta(days=365)
        tenancy_status = random.choices(["Active","Former"], weights=[0.85,0.15])[0]
        tenants.append({
            "TenantID": tid,
            "FullName": members_df.loc[members_df.MemberID==tid, "FullName"].values[0],
            "HouseID": h,
            "ApartmentID": apt_id,
            "LeaseStart": lease_start,
            "LeaseEnd": lease_end,
            "TenantResidencyStatus": tenancy_status,
            "LandlordID": ld["LandlordID"]
        })

# If remaining tenants, assign to random partly/fully rented houses
if tenant_index < len(tenant_candidates):
    remaining = tenant_candidates[tenant_index:]
    candidate_houses = [l for l in landlords if l["NumberOfTenants"]>0]
    for tid in remaining:
        ld = random.choice(candidate_houses)
        h = ld["HouseID"]
        hnum = int(h[1:])
        apt_counters[hnum] += 1
        apt_id = apartment_id(hnum, apt_counters[hnum])
        members_df.loc[members_df.MemberID==tid, "HouseID"] = h
        members_df.loc[members_df.MemberID==tid, "ApartmentID"] = apt_id
        lease_start = fake.date_between(datetime(2022,1,1), datetime(2024,5,1))
        lease_end = lease_start + timedelta(days=365)
        tenancy_status = random.choices(["Active","Former"], weights=[0.85,0.15])[0]
        tenants.append({
            "TenantID": tid,
            "FullName": members_df.loc[members_df.MemberID==tid, "FullName"].values[0],
            "HouseID": h,
            "ApartmentID": apt_id,
            "LeaseStart": lease_start,
            "LeaseEnd": lease_end,
            "TenantResidencyStatus": tenancy_status,
            "LandlordID": ld["LandlordID"]
        })

tenants_df = pd.DataFrame(tenants)

# Utilities
utilities = [
    {"UtilityID": "UT001", "UtilityName": "Electricity", "Description":"Monthly electricity charge", "BillingFrequency":"Monthly"},
    {"UtilityID": "UT002", "UtilityName": "Water", "Description":"Monthly water charge", "BillingFrequency":"Monthly"},
    {"UtilityID": "UT003", "UtilityName": "Waste Disposal", "Description":"Monthly waste charge", "BillingFrequency":"Monthly"},
    {"UtilityID": "UT004", "UtilityName": "Security", "Description":"Monthly security levy (per apartment/individual)", "BillingFrequency":"Monthly"},
    {"UtilityID": "UT005", "UtilityName": "Development Levy", "Description":"One-time lump-sum development levy", "BillingFrequency":"One-time"}
]
utilities_df = pd.DataFrame(utilities)

# Utility charges
charges = []
charge_counter = 1
for hrow in houses_df.itertuples():
    h = hrow.HouseID
    house_tenants = tenants_df[tenants_df.HouseID==h]
    for ut in utilities:
        uid = ut["UtilityID"]
        if uid == SECURITY_UTILITY_ID:
            # landlord-level security charge
            expected_amount = random.randint(4000,8000)
            charges.append({
                "ChargeID": charge_code(charge_counter),
                "UtilityID": uid,
                "HouseID": h,
                "ApartmentID": "",
                "ChargeAmount": float(expected_amount),
                "ChargeDate": CHARGE_MONTH.date(),
                "DueDate": (CHARGE_MONTH + timedelta(days=29)).date(),
                "ChargeMonth": CHARGE_MONTH.strftime("%Y-%m"),
                "PaymentStatus": "Pending"
            })
            charge_counter += 1
            for t in house_tenants.itertuples():
                expected_amount = random.randint(3000,7000)
                charges.append({
                    "ChargeID": charge_code(charge_counter),
                    "UtilityID": uid,
                    "HouseID": h,
                    "ApartmentID": t.ApartmentID,
                    "ChargeAmount": float(expected_amount),
                    "ChargeDate": CHARGE_MONTH.date(),
                    "DueDate": (CHARGE_MONTH + timedelta(days=29)).date(),
                    "ChargeMonth": CHARGE_MONTH.strftime("%Y-%m"),
                    "PaymentStatus": "Pending"
                })
                charge_counter += 1
        else:
            expected_amount = random.randint(8000,22000) if uid!="UT002" else random.randint(3000,8000)
            charges.append({
                "ChargeID": charge_code(charge_counter),
                "UtilityID": uid,
                "HouseID": h,
                "ApartmentID": "",
                "ChargeAmount": float(expected_amount),
                "ChargeDate": CHARGE_MONTH.date(),
                "DueDate": (CHARGE_MONTH + timedelta(days=29)).date(),
                "ChargeMonth": CHARGE_MONTH.strftime("%Y-%m"),
                "PaymentStatus": "Pending"
            })
            charge_counter += 1

charges_df = pd.DataFrame(charges)

# Payments simulation (~70% charges get payments)
payments = []
pay_counter = 1
for ch in charges_df.itertuples():
    if random.random() < 0.7:
        full_pay = random.random() < 0.6
        if full_pay:
            paid_amount = ch.ChargeAmount
        else:
            paid_amount = round(ch.ChargeAmount * random.uniform(0.3,0.9),2)
        payments.append({
            "PaymentID": payment_code(pay_counter),
            "ChargeID": ch.ChargeID,
            "HouseID": ch.HouseID,
            "ApartmentID": ch.ApartmentID,
            "PaidAmount": float(paid_amount),
            "PaymentDate": (CHARGE_MONTH + timedelta(days=random.randint(0,25))).date(),
            "PaymentMethod": random.choice(["Paystack","Bank Transfer","POS","Cash"]),
            "ReferenceNo": str(uuid.uuid4())[:18],
            "UtilityID": ch.UtilityID,
            "ExpectedAmount": ch.ChargeAmount
        })
        pay_counter += 1

payments_df = pd.DataFrame(payments)

# Update charges PaymentStatus as trigger would do
paid_sums = payments_df.groupby("ChargeID")["PaidAmount"].sum().reset_index().rename(columns={"PaidAmount":"TotalPaid"})
charges_df = charges_df.merge(paid_sums, how="left", on="ChargeID")
charges_df["TotalPaid"] = charges_df["TotalPaid"].fillna(0.0)
def status_from_amount(row):
    if row["TotalPaid"] >= row["ChargeAmount"]:
        return "Paid"
    elif row["TotalPaid"] > 0:
        return "Partially Paid"
    else:
        return "Pending"
charges_df["PaymentStatus"] = charges_df.apply(status_from_amount, axis=1)
charges_df = charges_df.drop(columns=["TotalPaid"])

# Ensure membership landlord house assignments
for ld in landlords_df.itertuples():
    members_df.loc[members_df.MemberID==ld.LandlordID, "HouseID"] = ld.HouseID
    members_df.loc[members_df.MemberID==ld.LandlordID, "ApartmentID"] = ""

# Save CSVs
members_df.to_csv(os.path.join(OUT_DIR,"membership.csv"), index=False)
landlords_df.to_csv(os.path.join(OUT_DIR,"landlords.csv"), index=False)
tenants_df.to_csv(os.path.join(OUT_DIR,"tenants.csv"), index=False)
utilities_df.to_csv(os.path.join(OUT_DIR,"utilities.csv"), index=False)
charges_df.to_csv(os.path.join(OUT_DIR,"utility_charges.csv"), index=False)
payments_df.to_csv(os.path.join(OUT_DIR,"payments.csv"), index=False)

# Try to display dataframes using caas_jupyter_tools if available
try:
    import caas_jupyter_tools as cjt
    cjt.display_dataframe_to_user("Membership (preview)", members_df.head(15))
    cjt.display_dataframe_to_user("Landlords (preview)", landlords_df.head(15))
    cjt.display_dataframe_to_user("Tenants (preview)", tenants_df.head(15))
    cjt.display_dataframe_to_user("Utility Charges (preview)", charges_df.head(20))
    cjt.display_dataframe_to_user("Payments (preview)", payments_df.head(20))
except Exception:
    pass

print("Saved CSVs to", OUT_DIR)
for f in sorted(os.listdir(OUT_DIR)):
    print("-", f)

CalledProcessError: Command '['C:\\Users\\DELL\\anaconda3\\python.exe', '-m', 'pip', 'install', 'faker']' returned non-zero exit status 1.

In [7]:
from faker import Faker
fake = Faker()
print(fake.name())

ModuleNotFoundError: No module named 'faker'

In [9]:
from faker import Faker
fake = Faker()
print(fake.name())

Kelly Elliott


In [11]:
# Retry: robust dataset generation with installation fallback for Faker
import sys, subprocess, os, uuid, random
from datetime import datetime, timedelta

# install faker if missing
try:
    from faker import Faker
except Exception:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "faker"])
    from faker import Faker

import pandas as pd
Faker = Faker
fake = Faker()
Faker.seed(42)
random.seed(42)

OUT_DIR = "/mnt/data/estate_sim"
os.makedirs(OUT_DIR, exist_ok=True)

# Parameters
NUM_HOUSES = 10
LANDLORDS_COUNT = 10
MEMBERS_TOTAL = 36
START_JOIN = datetime(2020,1,1)
END_JOIN = datetime(2024,6,1)
CHARGE_MONTH = datetime(2024,6,1)
SECURITY_UTILITY_ID = "UT004"

def house_id(n):
    return f"H{n:03d}"

def apartment_id(house_num, unit):
    return f"APT-{house_num:03d}{unit:02d}"

def member_code(n):
    return f"M{n:03d}"

def charge_code(n):
    return f"CH{n:03d}"

def payment_code(n):
    return f"P{n:04d}"

# Houses
houses = [{"HouseID": house_id(i), "HouseNumber": i} for i in range(1, NUM_HOUSES+1)]
houses_df = pd.DataFrame(houses)

# Members: landlords first
members = []
landlord_ids = []
for i in range(1, LANDLORDS_COUNT+1):
    mid = member_code(i)
    h = house_id(i)
    name = fake.name()
    email = f"{name.lower().replace(' ','.')}@estate.com"
    phone = fake.msisdn()[:13]
    join = fake.date_between(START_JOIN, END_JOIN)
    is_resident = random.choice([True, False])
    membership_status = random.choices(["Current","Former"], weights=[0.9,0.1])[0]
    members.append({
        "MemberID": mid,
        "FullName": name,
        "Role": "Landlord",
        "PhoneNumber": phone,
        "Email": email,
        "JoiningDate": join,
        "HouseID": h,
        "ApartmentID": "",
        "IsResident": is_resident,
        "MembershipStatus": membership_status
    })
    landlord_ids.append(mid)

next_id = LANDLORDS_COUNT + 1
num_tenants = 18
num_family = MEMBERS_TOTAL - LANDLORDS_COUNT - num_tenants

tenant_candidates = []
for i in range(num_tenants):
    mid = member_code(next_id)
    name = fake.name()
    email = f"{name.lower().replace(' ','.')}@estate.com"
    phone = fake.msisdn()[:13]
    join = fake.date_between(START_JOIN, END_JOIN)
    members.append({
        "MemberID": mid,
        "FullName": name,
        "Role": "Tenant",
        "PhoneNumber": phone,
        "Email": email,
        "JoiningDate": join,
        "HouseID": None,
        "ApartmentID": "",
        "IsResident": False,
        "MembershipStatus": random.choices(["Current","Former"], weights=[0.85,0.15])[0]
    })
    tenant_candidates.append(mid)
    next_id += 1

for i in range(num_family):
    mid = member_code(next_id)
    name = fake.name()
    email = f"{name.lower().replace(' ','.')}@estate.com"
    phone = fake.msisdn()[:13]
    join = fake.date_between(START_JOIN, END_JOIN)
    house_assigned = house_id(random.randint(1, NUM_HOUSES))
    members.append({
        "MemberID": mid,
        "FullName": name,
        "Role": "Family",
        "PhoneNumber": phone,
        "Email": email,
        "JoiningDate": join,
        "HouseID": house_assigned,
        "ApartmentID": "",
        "IsResident": True,
        "MembershipStatus": "Current"
    })
    next_id += 1

members_df = pd.DataFrame(members)

# Landlords derived table and tenant counts
landlords = []
assignment = ["Fully Owned","Partly Rented","Fully Owned","Fully Rented","Fully Owned",
              "Partly Rented","Partly Rented","Fully Rented","Fully Rented","Partly Rented"]

for idx, lid in enumerate(landlord_ids):
    house = house_id(idx+1)
    row = members_df.loc[members_df.MemberID==lid].iloc[0]
    is_res = row["IsResident"]
    membership_status = row["MembershipStatus"]
    ownership = assignment[idx]
    if ownership=="Fully Owned":
        num_tenants = 0
    elif ownership=="Partly Rented":
        num_tenants = random.randint(1,3)
    else:
        num_tenants = random.randint(4,5)
    if membership_status in ("Former",):
        ownership = "Fully Rented"
        num_tenants = random.randint(4,5)
    landlords.append({
        "LandlordID": lid,
        "FullName": row["FullName"],
        "PhoneNumber": row["PhoneNumber"],
        "Email": row["Email"],
        "HouseID": house,
        "StreetAddress": fake.street_address(),
        "IsResident": is_res,
        "OwnershipStatus": ownership,
        "MembershipStatus": membership_status,
        "NumberOfTenants": num_tenants
    })

landlords_df = pd.DataFrame(landlords)

# Allocate tenants per landlord
tenants = []
tenant_index = 0
apt_counters = {i:0 for i in range(1, NUM_HOUSES+1)}
for ld in landlords:
    h = ld["HouseID"]
    hnum = int(h[1:])
    n = ld["NumberOfTenants"]
    if n == 0:
        continue
    for unit in range(n):
        if tenant_index >= len(tenant_candidates):
            break
        tid = tenant_candidates[tenant_index]
        tenant_index += 1
        apt_counters[hnum] += 1
        apt_id = apartment_id(hnum, apt_counters[hnum])
        members_df.loc[members_df.MemberID==tid, "HouseID"] = h
        members_df.loc[members_df.MemberID==tid, "ApartmentID"] = apt_id
        lease_start = fake.date_between(datetime(2022,1,1), datetime(2024,5,1))
        lease_end = lease_start + timedelta(days=365)
        tenancy_status = random.choices(["Active","Former"], weights=[0.85,0.15])[0]
        tenants.append({
            "TenantID": tid,
            "FullName": members_df.loc[members_df.MemberID==tid, "FullName"].values[0],
            "HouseID": h,
            "ApartmentID": apt_id,
            "LeaseStart": lease_start,
            "LeaseEnd": lease_end,
            "TenantResidencyStatus": tenancy_status,
            "LandlordID": ld["LandlordID"]
        })

# If remaining tenants, assign to random partly/fully rented houses
if tenant_index < len(tenant_candidates):
    remaining = tenant_candidates[tenant_index:]
    candidate_houses = [l for l in landlords if l["NumberOfTenants"]>0]
    for tid in remaining:
        ld = random.choice(candidate_houses)
        h = ld["HouseID"]
        hnum = int(h[1:])
        apt_counters[hnum] += 1
        apt_id = apartment_id(hnum, apt_counters[hnum])
        members_df.loc[members_df.MemberID==tid, "HouseID"] = h
        members_df.loc[members_df.MemberID==tid, "ApartmentID"] = apt_id
        lease_start = fake.date_between(datetime(2022,1,1), datetime(2024,5,1))
        lease_end = lease_start + timedelta(days=365)
        tenancy_status = random.choices(["Active","Former"], weights=[0.85,0.15])[0]
        tenants.append({
            "TenantID": tid,
            "FullName": members_df.loc[members_df.MemberID==tid, "FullName"].values[0],
            "HouseID": h,
            "ApartmentID": apt_id,
            "LeaseStart": lease_start,
            "LeaseEnd": lease_end,
            "TenantResidencyStatus": tenancy_status,
            "LandlordID": ld["LandlordID"]
        })

tenants_df = pd.DataFrame(tenants)

# Utilities
utilities = [
    {"UtilityID": "UT001", "UtilityName": "Electricity", "Description":"Monthly electricity charge", "BillingFrequency":"Monthly"},
    {"UtilityID": "UT002", "UtilityName": "Water", "Description":"Monthly water charge", "BillingFrequency":"Monthly"},
    {"UtilityID": "UT003", "UtilityName": "Waste Disposal", "Description":"Monthly waste charge", "BillingFrequency":"Monthly"},
    {"UtilityID": "UT004", "UtilityName": "Security", "Description":"Monthly security levy (per apartment/individual)", "BillingFrequency":"Monthly"},
    {"UtilityID": "UT005", "UtilityName": "Development Levy", "Description":"One-time lump-sum development levy", "BillingFrequency":"One-time"}
]
utilities_df = pd.DataFrame(utilities)

# Utility charges
charges = []
charge_counter = 1
for hrow in houses_df.itertuples():
    h = hrow.HouseID
    house_tenants = tenants_df[tenants_df.HouseID==h]
    for ut in utilities:
        uid = ut["UtilityID"]
        if uid == SECURITY_UTILITY_ID:
            # landlord-level security charge
            expected_amount = random.randint(4000,8000)
            charges.append({
                "ChargeID": charge_code(charge_counter),
                "UtilityID": uid,
                "HouseID": h,
                "ApartmentID": "",
                "ChargeAmount": float(expected_amount),
                "ChargeDate": CHARGE_MONTH.date(),
                "DueDate": (CHARGE_MONTH + timedelta(days=29)).date(),
                "ChargeMonth": CHARGE_MONTH.strftime("%Y-%m"),
                "PaymentStatus": "Pending"
            })
            charge_counter += 1
            for t in house_tenants.itertuples():
                expected_amount = random.randint(3000,7000)
                charges.append({
                    "ChargeID": charge_code(charge_counter),
                    "UtilityID": uid,
                    "HouseID": h,
                    "ApartmentID": t.ApartmentID,
                    "ChargeAmount": float(expected_amount),
                    "ChargeDate": CHARGE_MONTH.date(),
                    "DueDate": (CHARGE_MONTH + timedelta(days=29)).date(),
                    "ChargeMonth": CHARGE_MONTH.strftime("%Y-%m"),
                    "PaymentStatus": "Pending"
                })
                charge_counter += 1
        else:
            expected_amount = random.randint(8000,22000) if uid!="UT002" else random.randint(3000,8000)
            charges.append({
                "ChargeID": charge_code(charge_counter),
                "UtilityID": uid,
                "HouseID": h,
                "ApartmentID": "",
                "ChargeAmount": float(expected_amount),
                "ChargeDate": CHARGE_MONTH.date(),
                "DueDate": (CHARGE_MONTH + timedelta(days=29)).date(),
                "ChargeMonth": CHARGE_MONTH.strftime("%Y-%m"),
                "PaymentStatus": "Pending"
            })
            charge_counter += 1

charges_df = pd.DataFrame(charges)

# Payments simulation (~70% charges get payments)
payments = []
pay_counter = 1
for ch in charges_df.itertuples():
    if random.random() < 0.7:
        full_pay = random.random() < 0.6
        if full_pay:
            paid_amount = ch.ChargeAmount
        else:
            paid_amount = round(ch.ChargeAmount * random.uniform(0.3,0.9),2)
        payments.append({
            "PaymentID": payment_code(pay_counter),
            "ChargeID": ch.ChargeID,
            "HouseID": ch.HouseID,
            "ApartmentID": ch.ApartmentID,
            "PaidAmount": float(paid_amount),
            "PaymentDate": (CHARGE_MONTH + timedelta(days=random.randint(0,25))).date(),
            "PaymentMethod": random.choice(["Paystack","Bank Transfer","POS","Cash"]),
            "ReferenceNo": str(uuid.uuid4())[:18],
            "UtilityID": ch.UtilityID,
            "ExpectedAmount": ch.ChargeAmount
        })
        pay_counter += 1

payments_df = pd.DataFrame(payments)

# Update charges PaymentStatus as trigger would do
paid_sums = payments_df.groupby("ChargeID")["PaidAmount"].sum().reset_index().rename(columns={"PaidAmount":"TotalPaid"})
charges_df = charges_df.merge(paid_sums, how="left", on="ChargeID")
charges_df["TotalPaid"] = charges_df["TotalPaid"].fillna(0.0)
def status_from_amount(row):
    if row["TotalPaid"] >= row["ChargeAmount"]:
        return "Paid"
    elif row["TotalPaid"] > 0:
        return "Partially Paid"
    else:
        return "Pending"
charges_df["PaymentStatus"] = charges_df.apply(status_from_amount, axis=1)
charges_df = charges_df.drop(columns=["TotalPaid"])

# Ensure membership landlord house assignments
for ld in landlords_df.itertuples():
    members_df.loc[members_df.MemberID==ld.LandlordID, "HouseID"] = ld.HouseID
    members_df.loc[members_df.MemberID==ld.LandlordID, "ApartmentID"] = ""

# Save CSVs
members_df.to_csv(os.path.join(OUT_DIR,"membership.csv"), index=False)
landlords_df.to_csv(os.path.join(OUT_DIR,"landlords.csv"), index=False)
tenants_df.to_csv(os.path.join(OUT_DIR,"tenants.csv"), index=False)
utilities_df.to_csv(os.path.join(OUT_DIR,"utilities.csv"), index=False)
charges_df.to_csv(os.path.join(OUT_DIR,"utility_charges.csv"), index=False)
payments_df.to_csv(os.path.join(OUT_DIR,"payments.csv"), index=False)

# Try to display dataframes using caas_jupyter_tools if available
try:
    import caas_jupyter_tools as cjt
    cjt.display_dataframe_to_user("Membership (preview)", members_df.head(15))
    cjt.display_dataframe_to_user("Landlords (preview)", landlords_df.head(15))
    cjt.display_dataframe_to_user("Tenants (preview)", tenants_df.head(15))
    cjt.display_dataframe_to_user("Utility Charges (preview)", charges_df.head(20))
    cjt.display_dataframe_to_user("Payments (preview)", payments_df.head(20))
except Exception:
    pass

print("Saved CSVs to", OUT_DIR)
for f in sorted(os.listdir(OUT_DIR)):
    print("-", f)

Saved CSVs to /mnt/data/estate_sim
- landlords.csv
- membership.csv
- payments.csv
- tenants.csv
- utilities.csv
- utility_charges.csv


In [13]:
import os
print(os.path.abspath("/mnt/data/estate_sim"))
os.listdir("/mnt/data/estate_sim")

C:\mnt\data\estate_sim


['landlords.csv',
 'membership.csv',
 'payments.csv',
 'tenants.csv',
 'utilities.csv',
 'utility_charges.csv']

In [15]:
import shutil, os

src_dir = "/mnt/data/estate_sim"
dst_dir = "C:/Users/DELL/Documents/estate_sim"

os.makedirs(dst_dir, exist_ok=True)

for file in os.listdir(src_dir):
    shutil.copy(os.path.join(src_dir, file), dst_dir)

print("Files copied to:", dst_dir)
print("Copied files:", os.listdir(dst_dir))


Files copied to: C:/Users/DELL/Documents/estate_sim
Copied files: ['landlords.csv', 'membership.csv', 'payments.csv', 'tenants.csv', 'utilities.csv', 'utility_charges.csv']
